In [25]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 7.1 MB/s eta 0:00:00


In [26]:
import pandas as pd
import pickle
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import nltk
from unidecode import unidecode

# Antes de executar esse código, baixe os recursos do NLTK executando:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [2]:
import pandas as pd
from sklearn.utils import resample

# Carregando os dados do CSV
dados1 = pd.read_csv('Construshow-Normalizados.csv')
dados = dados1[dados1['label'] != '']

#classe_majoritaria = dados[dados['label'] == 'PSGerencial']

# Lista com todas as classes únicas no target
#classes_unicas = dados['label'].unique()

# Lista para armazenar as classes oversampled
#classes_oversampled = []

# Aplicando oversampling para cada classe
'''for classe in classes_unicas:
    classe_dados = dados[dados['label'] == classe]
    classe_oversampled = resample(classe_dados,
                                  replace=True,  # Permitir duplicatas
                                  n_samples=50,  # Igualar ao tamanho da classe majoritária
                                  random_state=42)  # Reprodutibilidade
    classes_oversampled.append(classe_oversampled)

# Concatenando as classes oversampled
dados_oversampled = pd.concat(classes_oversampled)

# Exibindo o dataset resultante após o oversampling
print(dados_oversampled['label'].value_counts())'''

"for classe in classes_unicas:\n    classe_dados = dados[dados['label'] == classe]\n    classe_oversampled = resample(classe_dados,\n                                  replace=True,  # Permitir duplicatas\n                                  n_samples=50,  # Igualar ao tamanho da classe majoritária\n                                  random_state=42)  # Reprodutibilidade\n    classes_oversampled.append(classe_oversampled)\n\n# Concatenando as classes oversampled\ndados_oversampled = pd.concat(classes_oversampled)\n\n# Exibindo o dataset resultante após o oversampling\nprint(dados_oversampled['label'].value_counts())"

In [3]:
#dados_oversampled.to_csv('reduzido.csv')

In [4]:
dados.dropna(subset=['label'])
print(dados['label'].value_counts())

Infra                           191
Oracle Cloud                    177
Expedição                       177
Precificação/Tabela de Preço    166
Custo Médio/Aquisição           165
                               ... 
Roteiro de Logistica            108
Central de agendamento           96
Integração Forlog                96
Nota de Despesa                   1
Devolução de venda                1
Name: label, Length: 74, dtype: int64


In [5]:
#carregar planilha de tickets pré-classificados
#tickets = pd.read_csv("PetroShow.csv")
principais_colunas = ['text','label']
data = dados[principais_colunas]

data['Desc_Assunto'] = data['text']
data.dropna()

,text,label,Desc_Assunto
0,"boa tarde, tudo bem preço offline não está atu...",Preço off-line,"boa tarde, tudo bem preço offline não está atu..."
1,"boa tarde, tudo bem preço offline não está atu...",Preço off-line,"boa tarde, tudo bem preço offline não está atu..."
2,"boa tarde, tudo bem preço offline não está atu...",Preço off-line,"boa tarde, tudo bem preço offline não está atu..."
3,"boa tarde, tudo bem preço offline não está atu...",Preço off-line,"boa tarde, tudo bem preço offline não está atu..."
4,"boa tarde, tudo bem preço offline não está atu...",Preço off-line,"boa tarde, tudo bem preço offline não está atu..."
...,...,...,...
10692,"incluir caixa de aviso, que há operação sem sa...",Financeiro,"incluir caixa de aviso, que há operação sem sa..."
10693,"sistema caindo a conexão oi, boa tarde estamos...",Infra,"sistema caindo a conexão oi, boa tarde estamos..."
10694,reabrir chamado 479423 bom dia preciso reabiri...,Mobile Vendas,reabrir chamado 479423 bom dia preciso reabiri...
10695,devolução cancelou cartao da compra foi realiz...,Devolução de venda,devolução cancelou cartao da compra foi realiz...


In [27]:
custom_stopwords = [ "ao", "a", "de", "e", "um", "o", "não", "esta", "tela", "boa", "tarde", "em", "anexo", "cliente", "uma",
            "com", "na", "que", "esse", "problema", "segue", "para", "da", "situação", "do", "preciso", "essa", "estou",
            "conseguindo", "me", "isso", "no", "onde", "qual", "está", "sendo", "ticket", "usuários", "pois", "sem",
            "por", "conforme", "favor", "pelo", "atenciosamente", "os", "mas", "acesso", "pra", "data", "tudo", "bem",
            "estamos", "também", "caso", "fazer", "ele", "foi", "após", "nos", "dos", "campo", "aguardo", "já", "aparece",
            "este", "print", "nosso", "ou", "porém", "estão", "como", "exemplo", "temos", "bom", "dia", "seguinte", "as",
            "verificar", "precisamos", "mesmo", "nao", "seja", "novo", "hoje", "pode", "se", "alguns", "gostaria", "base",
            "ser", "informações", "abaixo", "mais", "quando", "tem", "ajuda", "das", "gerar", "dados", "viasoft", "eu",
            "forma", "att", "dando", "r", "urgente", "c", "código", "é", "tarde", ",", "também", "fica", "fazendo",
			"fica",  "só", "pela",  "resolver","dias", "estava",  "alteração", "poderia", "consta", "informação", "referente", "foram", "contato", "ltda", "ja", "correto", "olá", "via","todos", "são", "feita", "porque", "número", "somente", "feito", "alguma", "total", "desde", "ajuste","tentar", "todas", "entre", "obrigado", "valores",  "conseguimos", "gentileza", "alterar", "outra", "opção", "duas", "mesma",  "aparecendo", "ter", "seria",  "aqui", "momento", "retorno", "saber", "nas", "será", "mês", "agora", "podem", "suporte", "criar","consigo","nova", "ajustar", "realizar", "ver", "anydesk", "algumas", "porem", "apenas", "possível", "novamente", "ela", "assim", "dar",  "outro", "apresenta",  "sobre", "meu", "nome",  "algum", "auxilio", "excluir", "antes", "ainda", "poderiam", "hora",  "tentando", "existe", "gerando", "descrição", "solicito",  "aberto", "estar",  "depois",  "f", "ajudar", "vou", "dentro", "estab", "usuario", "até", "operação", "certo", "tenho", "pessoal", "fazendo", "d", "b","fica","tipo","fica","ocorre", "é", "corretamente","nossa", "quais",  "p",  "obrigada",  "acontecendo", "eles","essas","passo",  "clientes", "fico", "fizemos", "obs", "outros", "chamado", "gerado", "devido",  "agradeço", "faz", "diferença", "dessa","empresa", "passar", "sei",  "desta", "podemos", "deixar",  "lançar", "dois","nossos", "esses", "era", "informado", "nessa", "for",  "à", "seguintes",  "seu", "sempre",  "estabelecimento", "aparecem", "há", "final", "urgência", "tendo", "analista","filial", "incluir", "inclusão", "consegue", "necessário", "proceder", "identificar", "deveria", "cadastrar", "cada", "gera", "ontem", "padrão",  "validar",  "semana","continua", "módulo", "então", "vocês",  "solução", "precisa", "vai", "teria", "muito", "tempo", "nesse", "vamos", "neste",  "controle",  "vezes", "nem","solicitar", "auxiliar", "apresentando", "ficou", "testes", "através", "sim", "mensagem", "puxando", "permite", "origem", "fiz",  "desse", "acontece",  "nº", "nós", "puxa", "correta", "entender", "possui", "realizado",  "fui", "deu", "questão","dúvida", "ocorrendo", "deve", "ocorreu", "teste", "vários", "pessoa","informar",  "divergência",  "criação","qualquer", "consegui", "errada", "corrigir", "poder",  "aparecer", "kg", "saindo", "uso",  "parte", "gerou", "funcionando", "correção", "faço", "normal", "minha", "relação", "ai", "deste", "tivemos", "posso", "tentei",  "quanto", "deixa", "colocar", "mostra",  "nenhum", "vez", "tirar", "aos", "conseguem",  "ordem", "exclusão",  "errado", "prezados",  "nada", "aba"]

def remover_stopwords(texto):
    # Dividir a string em palavras individuais
    palavras = texto.split()

    # Filtrar as palavras que não são stopwords
    palavras_filtradas = [palavra for palavra in palavras if palavra.lower() not in custom_stopwords]

    # Reconstituir a string resultante
    texto_filtrado = ' '.join(palavras_filtradas)

    return texto_filtrado

def clean_text(text):
    stop_words =  custom_stopwords  # Lista de stopwords em português
    #stemmer = RSLPStemmer()  # Lematizador para português
    text = unidecode(text)  # Remover acentuação
    text = re.sub(r'[^\w\s]', ' ', text)  # Remover pontuações e caracteres especiais
    words = nltk.word_tokenize(text, language='portuguese')
    cleaned_words = [word.lower() for word in words if word.lower() not in stop_words]
    return ' '.join(cleaned_words)

In [28]:
#Limpeza de dados
data['Desc_BKP'] = data['Desc_Assunto']
data['Desc_Assunto'] = data['Desc_Assunto'].apply(lambda x: re.sub(r'http\S+', '', x))
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'
data['Desc_Assunto'] = data['Desc_Assunto'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
data['Desc_Assunto'] = data['Desc_Assunto'].str.lower()
data['Desc_Assunto'] = data['Desc_Assunto'].apply(remover_stopwords)
data['Desc_Assunto'] = data['Desc_Assunto'].apply(clean_text)
data['Desc_Assunto'] = data['Desc_Assunto'].str.replace("[0-9]", " ")
data['Desc_Assunto'] = data['Desc_Assunto'].apply(lambda x:' '.join(x.split()))
data

<ipython-input-28-4c99dd5eaabd>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Desc_Assunto'] = data['Desc_Assunto'].str.replace("[0-9]", " ")


,text,label,Desc_Assunto,Desc_BKP
0,"boa tarde, tudo bem preço offline não está atu...",Preço off-line,preco offline atualizando solicitada remarquei...,"tarde, preço offline atualizando solicitada. r..."
1,"boa tarde, tudo bem preço offline não está atu...",Preço off-line,preco offline atualizando solicitada remarquei...,"tarde, preço offline atualizando solicitada. r..."
2,"boa tarde, tudo bem preço offline não está atu...",Preço off-line,preco offline atualizando solicitada remarquei...,"tarde, preço offline atualizando solicitada. r..."
3,"boa tarde, tudo bem preço offline não está atu...",Preço off-line,preco offline atualizando solicitada remarquei...,"tarde, preço offline atualizando solicitada. r..."
4,"boa tarde, tudo bem preço offline não está atu...",Preço off-line,preco offline atualizando solicitada remarquei...,"tarde, preço offline atualizando solicitada. r..."
...,...,...,...,...
10692,"incluir caixa de aviso, que há operação sem sa...",Financeiro,caixa aviso salvar baixa operador feche salvar...,"caixa aviso, salvar baixa, operador feche salv..."
10693,"sistema caindo a conexão oi, boa tarde estamos...",Infra,sistema caindo conexao oi problemas conexao ca...,"sistema caindo conexão oi, problemas conexão, ..."
10694,reabrir chamado 479423 bom dia preciso reabiri...,Mobile Vendas,reabrir reabiri verificamos toda infrestutura ...,reabrir reabiri verificamos toda infrestutura ...
10695,devolução cancelou cartao da compra foi realiz...,Devolução de venda,devolucao cancelou cartao compra devolucao cai...,devolução cancelou cartao compra devolução cai...


In [29]:

vectorizer = CountVectorizer(lowercase=True, stop_words=custom_stopwords)
#data.dropna(subset=['Desc_Assunto'])
X = vectorizer.fit_transform(data["Desc_Assunto"])
y = data["label"]

In [30]:
# train the model
model = MultinomialNB()


In [31]:
model.fit(X, y)

MultinomialNB()

In [32]:
# save the model to a file
import pickle
with open("classification_model.pkl", "wb") as f:
    pickle.dump(model, f)

vec_file = 'vectorizer.pickle'
pickle.dump(vectorizer, open(vec_file, 'wb'))

In [33]:
loaded_vectorizer = pickle.load(open('vectorizer.pickle', 'rb'))


In [34]:
# Carregar modelo jÃ¡ treinado
with open('classification_model.pkl', 'rb') as f:
    modelo = pickle.load(f)

In [35]:
test_data_transformed = loaded_vectorizer.transform(["bom dia foi lançada a nfe no dia , não estou conseguindo autorizar....diz que ela foi duplicada. já tentei limpar a chave dela e não deu certo. preciso que vc autorize ela por gentileza, com urgência pois amanhã é o nosso fechamento. fico no aguardo de um retorno . obrigada nfe eco terminal"])
test_data_transformed

<1x8047 sparse matrix of type '<class 'numpy.int64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [61]:
teste =  [input()]

tickets_novos = pd.DataFrame(teste, columns=['assunto'])

tickets_novos['assunto'] = tickets_novos['assunto'].str.lower()

# remove URL's from train and test
tickets_novos['assunto'] = tickets_novos['assunto'].apply(lambda x: re.sub(r'http\S+', '', x))
###tickets_novos['assunto'] = tickets_novos['assunto'].apply(remover_stopwords)
#tickets_novos['assunto'] = tickets_novos['assunto'].apply(clean_text)

# test['clean_tweet'] = test['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))
# remove punctuation marks
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'

tickets_novos['assunto'] = tickets_novos['assunto'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
# test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

# convert text to lowercase
tickets_novos['assunto'] = tickets_novos['assunto'].str.lower()
# test['clean_tweet'] = test['clean_tweet'].str.lower()

# remove numbers
# tickets_novos['assunto'] = tickets_novos['assunto'].str.replace("[0-9]", " ")
# test['clean_tweet'] = test['clean_tweet'].str.replace("[0-9]", " ")

# remove whitespaces
tickets_novos['assunto'] = tickets_novos['assunto'].apply(lambda x: ' '.join(x.split()))
# test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ' '.join(x.split()))


tickets_novos['assunto'] = tickets_novos['assunto'].apply(lambda x: remover_stopwords(x))



test_data_transformed = loaded_vectorizer.transform(tickets_novos['assunto'])
predicted_categories = modelo.predict(test_data_transformed)
categoria = predicted_categories
print(categoria)
print(tickets_novos['assunto'])

urgente alteração na configuração de impostos todas as notas emitidas depois do dia 1807 ficaram sem a dedução do icms da base do cofins. conferindo a auditoria localizamos as seguintes alterações. precisamos saber quem fez essas alterações e se foi solicitado por alguém. precisamos também que seja apresentada uma solução para alterar esses registros, da forma como ficaram estão incorretos, e não temos pessoal e condições de alterar todas as venda que foram feitas no período só no dia de hoje 2707 que percebemos essa situação.
['Cadastro Tributação/Imposto']
0    configuração impostos notas emitidas 1807 fica...
Name: assunto, dtype: object


In [ ]:
ssem sistema bom dia estamos sem faturamento .from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
olá, boa tarde nossa contabilidade nos comunicou que ocorreram alguns erros no arquivo no ecf da loja graciki, referente ao mês . em anexo os registros em que ocorreram os erros. poderiam verificar esses erros e nos dar um retorno atenciosamente angelita erro ecf graciki